In [ ]:
# NBA Props Predictor - Data Exploration & Cleaning
# Notebook 02: Load, Clean, and Understand Your Data

# STEP 1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful")

# STEP 2: Load Data
print("\nLoading datasets...")

# Load historical data (NBA API)
historical_df = pd.read_csv('nba_player_game_logs.csv')
print(f"✓ Historical data loaded: {len(historical_df)} games")

# Load current season (ESPN) - if it exists
try:
    current_df = pd.read_csv('nba_current_season_2025_26.csv')
    print(f"✓ Current season loaded: {len(current_df)} games")
    
    # Combine datasets
    df = pd.concat([historical_df, current_df], ignore_index=True)
    print(f"✓ Combined dataset: {len(df)} total games")
except FileNotFoundError:
    print("⚠ Current season file not found, using historical data only")
    df = historical_df

# Convert GAME_DATE to datetime
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# STEP 3: Standardize Player Names (Remove Special Characters)
print("\n" + "="*60)
print("STANDARDIZING PLAYER NAMES")
print("="*60)

# Map special characters to standard ASCII
player_name_mapping = {
    'Nikola Jokić': 'Nikola Jokic',
    'Nikola Vučević': 'Nikola Vucevic',
    'Luka Dončić': 'Luka Doncic',
    'Kristaps Porziņģis': 'Kristaps Porzingis',
    'Dāvis Bertāns': 'Davis Bertans',
    'Bogdan Bogdanović': 'Bogdan Bogdanovic',
    'Nikola Jović': 'Nikola Jovic',
}

# Apply mapping
df['PLAYER_NAME'] = df['PLAYER_NAME'].replace(player_name_mapping)

print("✓ Player names standardized")
print("\nPlayers affected:")
for old_name, new_name in player_name_mapping.items():
    count = len(df[df['PLAYER_NAME'] == new_name])
    if count > 0:
        print(f"  {old_name} → {new_name}: {count} games")

# STEP 4: Fix Team Abbreviations and Match Opponent Stats
print("\n" + "="*60)
print("FIXING OPPONENT DEFENSIVE STATS")
print("="*60)

# Load team stats
try:
    team_stats = pd.read_csv('nba_team_stats.csv')
    print(f"✓ Loaded team stats: {len(team_stats)} team-seasons")
    
    # Check missing opponent stats before fixing
    missing_before = df['OPP_DEF_RATING'].isna().sum() if 'OPP_DEF_RATING' in df.columns else len(df)
    print(f"\nOpponent stats missing: {missing_before} games ({missing_before/len(df)*100:.1f}%)")
    
    # ESPN vs NBA API team abbreviation differences
    team_abbrev_mapping = {
        'BKN': 'BRK',   # Brooklyn Nets
        'CHA': 'CHO',   # Charlotte Hornets (sometimes CHH)
        'PHX': 'PHO',   # Phoenix Suns
        'SA': 'SAS',    # San Antonio Spurs
        'GS': 'GSW',    # Golden State Warriors
        'NO': 'NOP',    # New Orleans Pelicans
        'NY': 'NYK',    # New York Knicks
        'UTAH': 'UTA',  # Utah Jazz
        'WSH': 'WAS',   # Washington Wizards
    }
    
    # Create OPPONENT_FIXED column
    if 'OPPONENT' in df.columns:
        df['OPPONENT_FIXED'] = df['OPPONENT'].replace(team_abbrev_mapping)
    else:
        print("⚠ OPPONENT column not found, creating from MATCHUP")
        # Extract opponent from MATCHUP if needed
        def extract_opponent(matchup):
            if 'vs.' in str(matchup):
                return matchup.split('vs.')[-1].strip()
            elif '@' in str(matchup):
                return matchup.split('@')[-1].strip()
            return None
        df['OPPONENT'] = df['MATCHUP'].apply(extract_opponent)
        df['OPPONENT_FIXED'] = df['OPPONENT'].replace(team_abbrev_mapping)
    
    # Find correct team abbreviation column in team_stats
    team_col = None
    if 'TEAM_ABBREVIATION' in team_stats.columns:
        team_col = 'TEAM_ABBREVIATION'
    elif 'TEAM_ABBRV' in team_stats.columns:
        team_col = 'TEAM_ABBRV'
    else:
        # Create from team names using NBA API
        print("  Creating team abbreviations from team names...")
        from nba_api.stats.static import teams
        team_list = teams.get_teams()
        team_mapping_dict = {t['full_name']: t['abbreviation'] for t in team_list}
        team_stats['TEAM_ABBREVIATION'] = team_stats['TEAM_NAME'].map(team_mapping_dict)
        team_col = 'TEAM_ABBREVIATION'
    
    print(f"  Using team column: {team_col}")
    
    # Prepare opponent stats for merging
    opp_stats = team_stats[[team_col, 'SEASON', 'DEF_RATING', 'PACE']].copy()
    opp_stats.columns = ['OPPONENT_FIXED', 'SEASON', 'OPP_DEF_RATING_NEW', 'OPP_PACE_NEW']
    
    # Remove old opponent stats columns if they exist
    df = df.drop(columns=['OPP_DEF_RATING', 'OPP_PACE'], errors='ignore')
    
    # Merge with correct team abbreviations
    df = df.merge(opp_stats, on=['OPPONENT_FIXED', 'SEASON'], how='left')
    df = df.rename(columns={
        'OPP_DEF_RATING_NEW': 'OPP_DEF_RATING',
        'OPP_PACE_NEW': 'OPP_PACE'
    })
    
    # Check improvement
    still_missing = df['OPP_DEF_RATING'].isna().sum()
    fixed_count = missing_before - still_missing
    print(f"\n✓ Fixed {fixed_count} opponent matchings")
    print(f"  Still missing: {still_missing} games ({still_missing/len(df)*100:.1f}%)")
    
    # Show which opponents are still missing
    if still_missing > 0:
        print("\n  Opponents still missing stats:")
        missing_opponents = df[df['OPP_DEF_RATING'].isna()]['OPPONENT_FIXED'].value_counts()
        print(missing_opponents.head(10))
    
    # Fill remaining missing with league averages
    if still_missing > 0:
        league_avg_def = df['OPP_DEF_RATING'].mean()
        league_avg_pace = df['OPP_PACE'].mean()
        df['OPP_DEF_RATING'].fillna(league_avg_def, inplace=True)
        df['OPP_PACE'].fillna(league_avg_pace, inplace=True)
        print(f"\n  ✓ Filled remaining with league averages:")
        print(f"    DEF_RATING: {league_avg_def:.2f}")
        print(f"    PACE: {league_avg_pace:.2f}")
    
    print("\n✓ Opponent defensive ratings distribution:")
    print(df['OPP_DEF_RATING'].describe())
    
except FileNotFoundError:
    print("⚠ Team stats file not found, skipping opponent stats merge")

# STEP 5: Data Cleaning
print("\n" + "="*60)
print("DATA CLEANING")
print("="*60)

original_rows = len(df)

# Drop columns with >90% missing (not useful for modeling)
high_missing_threshold = 0.9
cols_to_check = df.columns
high_missing_cols = []

for col in cols_to_check:
    missing_pct = df[col].isna().sum() / len(df)
    if missing_pct > high_missing_threshold:
        high_missing_cols.append(col)

if high_missing_cols:
    df = df.drop(columns=high_missing_cols)
    print(f"✓ Dropped {len(high_missing_cols)} columns with >{high_missing_threshold*100}% missing:")
    for col in high_missing_cols:
        print(f"  - {col}")

# Fill shooting percentages with 0 (for games with 0 attempts)
shooting_cols = ['FG_PCT', 'FG3_PCT', 'FT_PCT']
for col in shooting_cols:
    if col in df.columns:
        df[col].fillna(0, inplace=True)
print(f"\n✓ Filled shooting percentages (0 for no attempts)")

# Drop rows missing critical stats
critical_cols = ['PTS', 'REB', 'AST', 'MIN', 'PLAYER_NAME', 'GAME_DATE']
existing_critical = [col for col in critical_cols if col in df.columns]
before_drop = len(df)
df = df.dropna(subset=existing_critical)
dropped_rows = before_drop - len(df)
if dropped_rows > 0:
    print(f"✓ Dropped {dropped_rows} rows missing critical stats ({dropped_rows/original_rows*100:.1f}%)")

# Fill remaining common missing values
if 'IS_HOME' in df.columns:
    df['IS_HOME'].fillna(0, inplace=True)

if 'PLUS_MINUS' in df.columns:
    df['PLUS_MINUS'].fillna(0, inplace=True)

if 'WL' in df.columns:
    df['WL'].fillna('L', inplace=True)

print(f"\n{'='*60}")
print("CLEANED DATASET SUMMARY")
print(f"{'='*60}")
print(f"Original rows: {original_rows:,}")
print(f"Cleaned rows: {len(df):,}")
print(f"Rows removed: {original_rows - len(df):,} ({(original_rows - len(df))/original_rows*100:.1f}%)")
print(f"\nMissing values remaining: {df.isnull().sum().sum()}")

remaining_missing = df.isnull().sum()
if remaining_missing.sum() > 0:
    print("\nColumns still with missing values:")
    missing_df = remaining_missing[remaining_missing > 0].sort_values(ascending=False)
    for col, count in missing_df.items():
        pct = (count / len(df)) * 100
        print(f"  {col}: {count} ({pct:.1f}%)")
else:
    print("\n✓ No missing values in critical columns!")

# STEP 6: Basic Data Info
print("\n" + "="*60)
print("DATASET OVERVIEW")
print("="*60)

print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Date range: {df['GAME_DATE'].min().date()} to {df['GAME_DATE'].max().date()}")
print(f"Unique players: {df['PLAYER_NAME'].nunique()}")
if 'SEASON' in df.columns:
    print(f"Seasons: {sorted(df['SEASON'].unique())}")

print("\nKey columns available:")
key_cols = ['PLAYER_NAME', 'GAME_DATE', 'PTS', 'REB', 'AST', 'MIN', 'OPPONENT', 
            'OPP_DEF_RATING', 'OPP_PACE', 'IS_HOME', 'WL']
for col in key_cols:
    status = "✓" if col in df.columns else "✗"
    print(f"  {status} {col}")

# STEP 7: Games Per Player
print("\n" + "="*60)
print("GAMES PER PLAYER")
print("="*60)

games_per_player = df.groupby('PLAYER_NAME').size().sort_values(ascending=False)
print(f"\nTop 10 players by games:")
print(games_per_player.head(10))

print(f"\nBottom 10 players by games:")
print(games_per_player.tail(10))

print(f"\nGames per player statistics:")
print(games_per_player.describe())

# Visualize
plt.figure(figsize=(14, 6))
games_per_player.head(20).plot(kind='bar', color='steelblue')
plt.title('Top 20 Players by Number of Games', fontsize=14, fontweight='bold')
plt.xlabel('Player', fontsize=12)
plt.ylabel('Number of Games', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# STEP 8: Statistical Summary
print("\n" + "="*60)
print("STATISTICAL SUMMARY")
print("="*60)

key_stats = ['MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 
             'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OPP_DEF_RATING', 'OPP_PACE']
available_stats = [stat for stat in key_stats if stat in df.columns]

print("\nDescriptive statistics:")
print(df[available_stats].describe().round(2))

# STEP 9: Points Distribution
print("\n" + "="*60)
print("POINTS DISTRIBUTION")
print("="*60)

print(f"\nPoints statistics:")
print(df['PTS'].describe())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['PTS'], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_title('Distribution of Points Scored', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Points')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df['PTS'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["PTS"].mean():.1f}')
axes[0].axvline(df['PTS'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["PTS"].median():.1f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(df['PTS'], vert=True)
axes[1].set_title('Points Box Plot', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Points')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# STEP 10: Top Performers
print("\n" + "="*60)
print("TOP PERFORMERS")
print("="*60)

# Calculate averages (min 20 games)
player_stats = df.groupby('PLAYER_NAME').agg({
    'PTS': ['mean', 'std', 'count'],
    'REB': 'mean',
    'AST': 'mean',
    'MIN': 'mean'
}).round(2)

player_stats.columns = ['PTS_AVG', 'PTS_STD', 'GAMES', 'REB_AVG', 'AST_AVG', 'MIN_AVG']
player_stats = player_stats[player_stats['GAMES'] >= 20].sort_values('PTS_AVG', ascending=False)

print("\nTop 15 scorers (min 20 games):")
print(player_stats.head(15))

# Visualize
plt.figure(figsize=(12, 6))
top_15 = player_stats.head(15)
plt.barh(range(len(top_15)), top_15['PTS_AVG'], xerr=top_15['PTS_STD'], 
         alpha=0.7, capsize=5, color='coral')
plt.yticks(range(len(top_15)), top_15.index)
plt.xlabel('Average Points Per Game', fontsize=12)
plt.title('Top 15 Scorers (with Std Dev)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

# STEP 11: Correlation Analysis
print("\n" + "="*60)
print("CORRELATION ANALYSIS")
print("="*60)

corr_stats = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'MIN', 'FGA', 'FG_PCT', 
              'FG3A', 'FG3_PCT', 'OPP_DEF_RATING', 'OPP_PACE']
corr_stats = [col for col in corr_stats if col in df.columns]

correlation_matrix = df[corr_stats].corr()

print("\nCorrelation with Points:")
print(correlation_matrix['PTS'].sort_values(ascending=False))

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# STEP 12: Save Cleaned Data
print("\n" + "="*60)
print("SAVING CLEANED DATA")
print("="*60)

output_file = 'nba_player_game_logs_cleaned.csv'
df.to_csv(output_file, index=False)
print(f"✓ Cleaned data saved to: {output_file}")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")

# STEP 13: Final Summary
print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)

print(f"""
Dataset Summary:
  • Total games: {len(df):,}
  • Unique players: {df['PLAYER_NAME'].nunique()}
  • Date range: {df['GAME_DATE'].min().date()} to {df['GAME_DATE'].max().date()}
  • Avg points: {df['PTS'].mean():.1f} ± {df['PTS'].std():.1f}
  • Avg rebounds: {df['REB'].mean():.1f} ± {df['REB'].std():.1f}
  • Avg assists: {df['AST'].mean():.1f} ± {df['AST'].std():.1f}

Data Quality:
  • Missing values: {df.isnull().sum().sum()}
  • Opponent stats coverage: {(1 - df['OPP_DEF_RATING'].isna().sum()/len(df))*100:.1f}%
  
✓ Ready for feature engineering!
""")

print("\nNext step: Run 03_feature_engineering.ipynb")

✓ Imports successful

Loading datasets...


FileNotFoundError: [Errno 2] No such file or directory: 'data/nba_player_game_logs.csv'

In [ ]:
# Check remaining missing values
print("\n" + "="*60)
print("REMAINING MISSING VALUES BREAKDOWN")
print("="*60)

missing = df.isnull().sum()
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percentage': (missing.values / len(df) * 100).round(2)
})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

print(f"\nColumns with missing values:")
print(missing_df.to_string(index=False))

# Categorize by importance
critical_stats = ['PTS', 'REB', 'AST', 'MIN']
important_stats = ['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'STL', 'BLK', 'TOV']
contextual_stats = ['OPP_DEF_RATING', 'OPP_PACE', 'IS_HOME', 'OPPONENT']
non_essential = ['VIDEO_AVAILABLE', 'SEASON_ID', 'Player_ID']

print(f"\n{'='*60}")
print("IMPACT ASSESSMENT")
print(f"{'='*60}")

critical_missing = missing_df[missing_df['Column'].isin(critical_stats)]
if len(critical_missing) > 0:
    print(f"CRITICAL stats missing: {critical_missing['Missing'].sum()}")
    print(critical_missing.to_string(index=False))
else:
    print(f"CRITICAL stats (PTS, REB, AST, MIN): 0 missing")

important_missing = missing_df[missing_df['Column'].isin(important_stats)]
if len(important_missing) > 0:
    print(f"\nIMPORTANT stats missing: {important_missing['Missing'].sum()}")
    print(important_missing.to_string(index=False))
else:
    print(f"IMPORTANT stats (shooting, defense): 0 missing")

contextual_missing = missing_df[missing_df['Column'].isin(contextual_stats)]
if len(contextual_missing) > 0:
    print(f"\nCONTEXTUAL stats missing: {contextual_missing['Missing'].sum()}")
    print(contextual_missing.to_string(index=False))
else:
    print(f"CONTEXTUAL stats (opponent, location): 0 missing")

print(f"\n{'='*60}")
print(f"READY FOR MODELING: {'YES' if len(critical_missing) == 0 else 'NO - FIX CRITICAL STATS FIRST'}")
print(f"{'='*60}")